In [1286]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [1287]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import sys
sys.path.append('../..') # Adiciona o diretório superior ao caminho de importação para acessar funções utilitárias
from functions.db_inserts import * # Importa funções para inserção de dados no banco de dados
from functions.aws_functions import * # Importa funções relacionadas à AWS
from functions.select_cols_functions import * # Importa funções para seleção e processamento de colunas
from tensorflow.keras.regularizers import l2

In [1288]:
env_mode = 'dev' # Define o modo de ambiente: 'dev' para desenvolvimento, 'prd' para produção

### Lendo arquivo fonte

In [1289]:
bucket_name = "adoptai-trusted-dev" # Nome do bucket S3 de onde os dados serão carregados

In [1290]:
if env_mode == 'dev':
    # Modo de desenvolvimento: carrega e processa os dados localmente
    df_trusted = pd.read_csv('../../../data/train_2_classes.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = ['train_2_classes.csv', df_trusted]
elif env_mode == 'prd':
    # Modo de produção: carrega e processa os dados a partir do S3
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Parâmetros de gravação

In [1291]:
nome_fonte = s3[0] # Armazena o nome do arquivo de dados utilizado
nome_modelo = 'Sequential (Rede Neural)' # Define o nome do modelo
parameters = {
    '1_layer_neuro': [
        10
        # 400, 
        # 500, 
        # 600
    ], # Define a quantidade de neurônios na primeira camada
}

### Preparativos para executar o modelo

In [1292]:
# Separar as features (x) e os rótulos (y)
x = df_trusted.iloc[:, :-1] # Seleciona todas as colunas, exceto a última, como features
y = df_trusted.iloc[:, -1] # Seleciona a última coluna como rótulo

In [1293]:
# Dividir os dados em conjuntos de treinamento e teste
x_train_full, x_test, y_train_full, y_test = train_test_split(x, y, test_size=0.05, random_state=42)

In [1294]:
x_valid, x_train = x_train_full[:int(len(x_train_full) * 0.1)], x_train_full[int(len(x_train_full) * 0.1):]
y_valid, y_train = y_train_full[:int(len(y_train_full) * 0.1)], y_train_full[int(len(y_train_full) * 0.1):]

In [1295]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [1296]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(12819, 21)
(1424, 21)
(750, 21)


###  Rodando modelo Sequencial

In [1297]:
# Listas para armazenar métricas de desempenho e tempos de execução
accuracy = []
precision = []
recall = []
f1 = []

start_times = []
end_times = []

In [1298]:
# start_times.append(time.time()) # Registra o tempo de início

# Inicializa o modelo com os parâmetros atuais
model = keras.models.Sequential([
    # keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(16, activation="relu", name='input'),
    keras.layers.Dense(10, activation="relu", name='1aCamadaEscondida'),
    keras.layers.Dense(5, activation="relu", name='2aCamadaEscondida'),
    keras.layers.Dense(2, activation="softmax", name='output')
])

model.compile(loss="sparse_categorical_crossentropy",
            optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
            metrics=["accuracy"])

history = model.fit(x_train,
                y_train,
                epochs=5,
                validation_data=(x_valid, y_valid)
                )

# Fazer previsões no conjunto de teste
y_pred = model.predict(x_test)
print('-' * 50)
model.evaluate(x_test, y_test)

Epoch 1/5
401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.4944 - loss: 7214.6973 - val_accuracy: 0.4979 - val_loss: 6544.0000
Epoch 2/5
401/401 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - accuracy: 0.4944 - loss: 6411.0166 - val_accuracy: 0.4979 - val_loss: 5770.0566
Epoch 3/5
401/401 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - accuracy: 0.4944 - loss: 5640.7324 - val_accuracy: 0.4979 - val_loss: 5027.0703
Epoch 4/5
401/401 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.4944 - loss: 4901.3877 - val_accuracy: 0.4979 - val_loss: 4314.3506
Epoch 5/5
401/401 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - accuracy: 0.4944 - loss: 4193.1250 - val_accuracy: 0.4979 - val_loss: 3642.5640
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
--------------------------------------------------
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.4911 - loss: 3758.5220
